## Integration of Google BigQuery DB with Databricks to enable seamless querying, processing, and analysis of large datasets.

## Google BigQuery integration with Databricks to enable seamless querying, processing, and analysis of large datasets.

Applications: ETL workflows across BigQuery & Databricks  * Analyzing large datasets using Spark & ML tools     *   Querying BigQuery data using SQL in Databricks   *   Optimizing cloud-based data pipelines

## Other applications:  Real-time analytics, streaming data from BigQuery into Databricks  or  Machine Learning, by using Databricks MLflow on BigQuery datasets, ETL Pipelines for extracting, transforming, and loading data from BigQuery, BI Dashboards  for Power BI or Tableau integration with Databricks & BigQuery


In [ ]:
#  Installing Required Packages
%pip install google-cloud-bigquery pandas pybigquery


In [ ]:
# Set Up Authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/dbfs/FileStore/service_key.json"

# or just set credentials directly in Databricks
spark.conf.set("google.cloud.auth.service.account.json.keyfile", "/dbfs/FileStore/service_key.json")

In [ ]:
# Load BigQuery Data into a Pandas DataFrame
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()
query = """
SELECT journal, COUNT(*) AS Freq
FROM `covid_research_db.metadata_2020_07_01`
WHERE journal IS NOT NULL
GROUP BY journal
ORDER BY Freq DESC
LIMIT 5
"""

df = client.query(query).to_dataframe()
display(df)

In [ ]:
# Load BigQuery Data into a Spark DataFrame
df_spark = spark.read.format("bigquery") \
    .option("credentialsFile", "/dbfs/FileStore/service_key.json") \
    .option("project", "your-gcp-project-id") \
    .option("dataset", "covid_research_db") \
    .option("table", "metadata_2020_07_01") \
    .load()

df_spark.createOrReplaceTempView("bigquery_table")

df_spark.show(5)

In [ ]:
#  Run SQL Query in Databricks
result = spark.sql("""
SELECT journal, COUNT(*) AS Freq
FROM bigquery_table
GROUP BY journal
ORDER BY Freq DESC
LIMIT 5
""")

In [ ]:
#  Write Data Back to BigQuery: 
df_spark.write.format("bigquery") \
    .option("credentialsFile", "/dbfs/FileStore/service_key.json") \
    .option("project", "your-gcp-project-id") \
    .option("dataset", "covid_research_db") \
    .option("table", "processed_results") \
    .mode("overwrite") \
    .save()
